# Notebook Cinema Emotion

__Presentation__: This notebook is aimed at extracting, transforming and loading the data from IMDB and TMDB in the MySQL server Database.

__Prerequisites__: Docker image jupyter/all-spark-notebook:latest (Spark, Python 3)

__Imports__: 

__Input Data__: [imdb](https://www.imdb.com/)

__Autors__: [David URBAN](https://www.linkedin.com/in/david-urban-703b9813a/)

__Contact Email__: david.urban@labom2iformation.fr

__Create at__: 28/08/2023



In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install NRCLex

Note: you may need to restart the kernel to use updated packages.


In [4]:
import mysql.connector
from mysql.connector import Error

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nrclex import NRCLex

import string
import pandas as pd

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType
from pyspark.sql.functions import explode,array,split,collect_list, col

import DB_credentials

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
appName = "PySpark Emotion Analysis"
master = "local"

spark = SparkSession.builder.master(master).appName(appName).getOrCreate()

In [6]:
def get_movie_id_list():
    try:
        connection = mysql.connector.connect(host=DB_credentials.host,
                                             database=DB_credentials.database,
                                             user=DB_credentials.user,
                                             password=DB_credentials.password)
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = connection.cursor()
            query = "SELECT id FROM movie"
            # Create a pandas dataframe
            pdf = pd.read_sql(query, con=connection)

            cursor.close()
            connection.close()
            print("MySQL connection is closed")

    except Error as e:
        print("Error while connecting to MySQL", e)
        pdf=[]
    return pdf

def get_analysis_id_list():
    try:
        connection = mysql.connector.connect(host=DB_credentials.host,
                                             database=DB_credentials.database,
                                             user=DB_credentials.user,
                                             password=DB_credentials.password)
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = connection.cursor()
            cursor.execute(create_table_query('positive_analysis'))
            connection.commit()
            query = "SELECT movie_id FROM positive_analysis"
            # Create a pandas dataframe
            pdf = pd.read_sql(query, con=connection)

            cursor.close()
            connection.close()
            print("MySQL connection is closed")

    except Error as e:
        print("Error while connecting to MySQL", e)
        pdf=[]
    return pdf

#movie_id_df = get_analysis_id_list() 
#movie_id_list = movie_id_df["movie_id"].values.tolist()
#movie_id_list

In [7]:


def get_reviews_and_score_from_movie_id(movie_id):
    try:
        connection = mysql.connector.connect(host=DB_credentials.host,
                                             database=DB_credentials.database,
                                             user=DB_credentials.user,
                                             password=DB_credentials.password)
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = connection.cursor()
            query = "SELECT score, text FROM review WHERE movie_id = "+ str(movie_id)
            # Create a pandas dataframe
            pdf = pd.read_sql(query, con=connection)

            cursor.close()
            connection.close()
            print("MySQL connection is closed")
            #print(pdf)

    except Error as e:
        print("Error while connecting to MySQL", e)
        pdf=[]
    return pdf

#pdf = get_reviews_and_score_from_movie_id(9)

In [8]:
#pdf.info()

In [9]:
# Convert Pandas dataframe to spark DataFrame

#mySchema = StructType([ StructField("score", IntegerType(), True)\
#                       ,StructField("text", StringType(), True)])

#Create DataFrame by changing schema
#df = spark.createDataFrame(pdf,schema=mySchema)
#df.show()
#df.printSchema()

In [10]:
#pos_reviews=df.filter(col('score') >5)
#neg_reviews=df.filter(col('score') <=5)
#pos_reviews.show()

In [11]:
#pos_text = pos_reviews.select('text').rdd
#neg_text = neg_reviews.select('text').rdd
#neg_text.collect()

### lower case

In [12]:
#pos_lower_casetext = pos_text.map(lambda x:x['text'].lower())
#neg_lower_casetext = neg_text.map(lambda x:x['text'].lower())
#pos_lower_casetext.collect()[0]

###  sentence tokenization

In [13]:

# Function to perform Sentence tokeniaztion
def sent_TokenizeFunct(x):
    return nltk.sent_tokenize(x)

#pos_sentencetokenization_rdd = pos_lower_casetext.map(sent_TokenizeFunct)
#neg_sentencetokenization_rdd = neg_lower_casetext.map(sent_TokenizeFunct)
#pos_sentencetokenization_rdd.collect()


### word Tokenize

In [14]:
def word_TokenizeFunct(x):
    splitted = [word for line in x for word in line.split()]
    return splitted

#pos_wordtokenization_rdd = pos_sentencetokenization_rdd.map(word_TokenizeFunct)
#neg_wordtokenization_rdd = neg_sentencetokenization_rdd.map(word_TokenizeFunct)
#pos_wordtokenization_rdd.collect()[0]

### Remove punctuation

In [15]:
# Remove Punctuation marks

def removePunctuationsFunct(x):
    list_punct=string.punctuation
    list_punct=list(list_punct.replace("\'",""))
    filtered = [''.join(c for c in s if c not in list_punct) for s in x] 
    filtered_space = [s for s in filtered if s] #remove empty space 
    return filtered_space

#pos_rmvPunctRDD = pos_wordtokenization_rdd.map(removePunctuationsFunct)
#neg_rmvPunctRDD = neg_wordtokenization_rdd.map(removePunctuationsFunct)
#pos_rmvPunctRDD.collect()[0]

### Remove stop words

In [16]:
from nltk.corpus import stopwords
#stopwords.words('english')

In [17]:
def removeStopWordsFunct(x):
    from nltk.corpus import stopwords
    stop_words=set(stopwords.words('english'))
    filteredSentence = [w for w in x if not w in stop_words]
    return filteredSentence

#pos_stopwordRDD = pos_rmvPunctRDD.map(removeStopWordsFunct)
#neg_stopwordRDD = neg_rmvPunctRDD.map(removeStopWordsFunct)
#pos_stopwordRDD.collect()[0]

### Word lemmatization

In [18]:
# Perform Lemmatization

def lemma(x):

    lemmatizer = WordNetLemmatizer()

    final_rdd = [lemmatizer.lemmatize(s) for s in x]
    return final_rdd

#pos_lem_wordsRDD = pos_stopwordRDD.map(lemma)
#neg_lem_wordsRDD = neg_stopwordRDD.map(lemma)

#pos_lem_wordsRDD.collect()[0]

### word count

In [19]:
def top_ten_word_count(x):
    words_per_comment=x.collect()
    words = [item for row in words_per_comment for item in row]
    wordcountdict={w: words.count(w) for w in words}
    top10words = dict(sorted(wordcountdict.items(), key = lambda x: x[1], reverse = True)[:10])
    return top10words

#neg_top10words = top_ten_word_count(neg_lem_wordsRDD)
#pos_top10words = top_ten_word_count(pos_lem_wordsRDD)
#pos_top10words

### bigram count

In [20]:
def RDD_to_Dataframe(x):
    ##Create DataFrame from RDD
    try:
        new_df = x.map(lambda y: (y, )).toDF(["features"])
        tokenized_df = new_df.withColumn("values", explode("features")).select("values")

        ## Create Final DataFrme 
        final_df = tokenized_df.select(collect_list("values").alias("listed_data"))
        final_df.persist()
    except ValueError:
        final_df = None
    return final_df

#neg_df = RDD_to_Dataframe(neg_lem_wordsRDD)
#pos_df = RDD_to_Dataframe(pos_lem_wordsRDD)
#neg_df

In [21]:
from pyspark.ml.feature import NGram
## Bigram
def top_ten_bigram_count(x):
    if x is not None:
        bigram = NGram(n=2, inputCol="listed_data", outputCol="bigrams")

        bigramDataFrame = bigram.transform(x)

        bigram_FinalDataFrame = bigramDataFrame.withColumn("bigram_final",explode("bigrams")).select("bigram_final")
        bigram_list = bigram_FinalDataFrame.rdd.map(lambda y:y['bigram_final'])
        bigram_list=bigram_list.collect()
        bigram_countdict={w: bigram_list.count(w) for w in bigram_list}
        top10bigram = dict(sorted(bigram_countdict.items(), key = lambda y: y[1], reverse = True)[:10])
    else :
        top10bigram = {}
    return top10bigram

#neg_top10bigram = top_ten_bigram_count(neg_df)
#pos_top10bigram = top_ten_bigram_count(pos_df)
#neg_top10bigram

### trigram count

In [22]:
## Trigram

def top_ten_trigram_count(x):
    if x is not None:
        trigram = NGram(n=3, inputCol="listed_data", outputCol="trigrams")

        trigramDataFrame = trigram.transform(x)

        trigram_FinalDataFrame = trigramDataFrame.withColumn("trigram_final",explode("trigrams")).select("trigram_final")
        trigram_list = trigram_FinalDataFrame.rdd.map(lambda y:y['trigram_final'])
        trigram_list=trigram_list.collect()
        trigram_countdict={w: trigram_list.count(w) for w in trigram_list}
        top10trigram = dict(sorted(trigram_countdict.items(), key = lambda y: y[1], reverse = True)[:10])
    else:
        top10trigram = {}
    return top10trigram

#neg_top10trigram = top_ten_trigram_count(neg_df)
#pos_top10trigram = top_ten_trigram_count(pos_df)
#pos_top10trigram

### Emotion classification

In [23]:
def emotion_classification(x):

    emotion_list = [(word,NRCLex(word)) for word in x]
    top_emotion_list = [(x[0], x[1].top_emotions) for x in emotion_list ]
    return top_emotion_list

#pos_top_emotion_list = pos_lem_wordsRDD.map(emotion_classification)
#neg_top_emotion_list = neg_lem_wordsRDD.map(emotion_classification)
#pos_top_emotion_list.collect()[0]

In [24]:
def emotion_classification_weight(x):

    emotion_list = [NRCLex(word).raw_emotion_scores for word in x]
    
    return emotion_list

#pos_top_emotion_list = pos_lem_wordsRDD.map(emotion_classification_weight)
#neg_top_emotion_list = neg_lem_wordsRDD.map(emotion_classification_weight)
#pos_top_emotion_list.collect()[0]

In [25]:
def get_emotion_distribution(x):
    res = {'disgust':0, 'fear':0, 'anticipation':0, 'joy':0, 'surprise':0, 
            'sadness':0, 'anger':0, 'trust':0, 'positive':0, 'negative':0}
    for i in x:
        for j in i:
            res = {k: res.get(k, 0) + j.get(k, 0) for k in set(res) | set(j)}
    try:
        positive = res.pop('positive')
    except KeyError:
        positive = 0
    try:
        negative = res.pop('negative')
    except KeyError:
        negative = 0
    if sum(res.values())>0:
        res = {k: v/sum(res.values()) for k, v in res.items()}
    try:
        res['positive']= positive/(positive+negative)
        res['negative']= negative/(positive+negative)
    except ZeroDivisionError:
        res['positive']= 0
        res['negative']= 0
    return res
#pos_emotion_distribution=get_emotion_distribution(pos_top_emotion_list.collect())
#neg_emotion_distribution=get_emotion_distribution(neg_top_emotion_list.collect())
#neg_emotion_distribution

In [26]:
def insert_top_ten_word_ensemble_in_dict(dictionnary,top10ensemble, word_ensemble_abbreviation):
    
    top10words_to_save= {word_ensemble_abbreviation+str(i): None for i in range(10)}
    top10words_to_save.update({word_ensemble_abbreviation+str(i): key for i, key in enumerate(top10ensemble.keys())})
    top10words_weights_to_save= {word_ensemble_abbreviation+"w"+str(i): None for i in range(10)}
    top10words_weights_to_save.update({word_ensemble_abbreviation+"w"+str(i): top10ensemble[key] for i, key in enumerate(top10ensemble.keys())})
    dictionnary.update(top10words_to_save)
    dictionnary.update(top10words_weights_to_save)
    return dictionnary

def analyse_reviews(text,movie_id):
    # text cleaning
    lower_casetext = text.map(lambda x:x['text'].lower())
    sentencetokenization_rdd = lower_casetext.map(sent_TokenizeFunct)
    wordtokenization_rdd = sentencetokenization_rdd.map(word_TokenizeFunct)
    rmvPunctRDD = wordtokenization_rdd.map(removePunctuationsFunct)
    stopwordRDD = rmvPunctRDD.map(removeStopWordsFunct)
    lem_wordsRDD = stopwordRDD.map(lemma)
    # word count
    top10words = top_ten_word_count(lem_wordsRDD)
    df = RDD_to_Dataframe(lem_wordsRDD)
    top10bigram = top_ten_bigram_count(df)
    top10trigram = top_ten_trigram_count(df)
    # emotion analysis
    top_emotion_list = lem_wordsRDD.map(emotion_classification_weight)
    emotion_distribution=get_emotion_distribution(top_emotion_list.collect())
    # data to save
    output_to_save = {'movie_id':movie_id}
    output_to_save.update(emotion_distribution)
    output_to_save = insert_top_ten_word_ensemble_in_dict(output_to_save,top10words,'m')
    output_to_save = insert_top_ten_word_ensemble_in_dict(output_to_save,top10bigram,'b')
    output_to_save = insert_top_ten_word_ensemble_in_dict(output_to_save,top10trigram,'t')
    return output_to_save

#neg_output = analyse_reviews(neg_text,1)
#pos_output = analyse_reviews(pos_text,1)
#neg_output


In [27]:
#neg_output.keys()

In [28]:

list_columns=['movie_id', 
 'disgust', 'fear', 'anticipation', 'joy', 'surprise', 
 'sadness', 'anger', 'trust', 'positive', 'negative', 
 'm0', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 
 'mw0', 'mw1', 'mw2', 'mw3', 'mw4', 'mw5', 'mw6', 'mw7', 'mw8', 'mw9', 
 'b0', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9', 
 'bw0', 'bw1', 'bw2', 'bw3', 'bw4', 'bw5', 'bw6', 'bw7', 'bw8', 'bw9', 
 't0', 't1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 
 'tw0', 'tw1', 'tw2', 'tw3', 'tw4', 'tw5', 'tw6', 'tw7', 'tw8', 'tw9']

def create_table_query(table_name):
    return ('CREATE TABLE IF NOT EXISTS '+ table_name + ' '
                 '(movie_id INT UNIQUE, '
                 'FOREIGN KEY (movie_id) '
                 'REFERENCES movie(id), '
                 'disgust FLOAT, fear FLOAT, sadness FLOAT, anger FLOAT, '
                 'anticipation FLOAT, joy FLOAT, surprise FLOAT, trust FLOAT, '
                 'positive FLOAT, negative FLOAT, '
                 'm0 VARCHAR(255), m1 VARCHAR(255), m2 VARCHAR(255), m3 VARCHAR(255), m4 VARCHAR(255), '
                 'm5 VARCHAR(255), m6 VARCHAR(255), m7 VARCHAR(255), m8 VARCHAR(255), m9 VARCHAR(255), '
                 'mw0 INT, mw1 INT, mw2 INT, mw3 INT, mw4 INT, mw5 INT, mw6 INT, mw7 INT, mw8 INT, mw9 INT, '
                 'b0  VARCHAR(255), b1 VARCHAR(255), b2 VARCHAR(255), b3 VARCHAR(255), b4 VARCHAR(255), '
                 'b5 VARCHAR(255), b6 VARCHAR(255), b7 VARCHAR(255), b8 VARCHAR(255), b9 VARCHAR(255), '
                 'bw0 INT, bw1 INT, bw2 INT, bw3 INT, bw4 INT, bw5 INT, bw6 INT, bw7 INT, bw8 INT, bw9 INT, '
                 't0 VARCHAR(255), t1 VARCHAR(255), t2 VARCHAR(255), t3 VARCHAR(255), t4 VARCHAR(255), '
                 't5 VARCHAR(255), t6 VARCHAR(255), t7 VARCHAR(255), t8 VARCHAR(255), t9 VARCHAR(255), '
                 'tw0 INT, tw1 INT, tw2 INT, tw3 INT, tw4 INT, tw5 INT, tw6 INT, tw7 INT, tw8 INT, tw9 INT);')

def add_table_query(table_name):
    return ('INSERT INTO '+ table_name + ' '
               '(movie_id, disgust, fear, sadness, anger, anticipation, joy, surprise, trust, positive, negative, '
               'm0, m1, m2, m3, m4, m5, m6, m7, m8, m9, mw0, mw1, mw2, mw3, mw4, mw5, mw6, mw7, mw8, mw9, '
               'b0, b1, b2, b3, b4, b5, b6, b7, b8, b9, bw0, bw1, bw2, bw3, bw4, bw5, bw6, bw7, bw8, bw9, '
               't0, t1, t2, t3, t4, t5, t6, t7, t8, t9, tw0, tw1, tw2, tw3, tw4, tw5, tw6, tw7, tw8, tw9) '
               'VALUES (' + '%s, '*70 + '%s)')

def update_table_query(table_name):
    return ('UPDATE '+ table_name + ' '
            'SET disgust = %s, fear = %s, sadness = %s, anger = %s, '
            'anticipation = %s, joy = %s, surprise = %s, trust = %s, positive = %s, negative = %s, '
            'm0 = %s, m1 = %s, m2 = %s, m3 = %s, m4 = %s, m5 = %s, m6 = %s, m7 = %s, m8 = %s, m9 = %s, '
            'mw0 = %s, mw1 = %s, mw2 = %s, mw3 = %s, mw4 = %s, mw5 = %s, mw6 = %s, mw7 = %s, mw8 = %s, mw9 = %s, '
            'b0 = %s, b1 = %s, b2 = %s, b3 = %s, b4 = %s, b5 = %s, b6 = %s, b7 = %s, b8 = %s, b9 = %s, '
            'bw0 = %s, bw1 = %s, bw2 = %s, bw3 = %s, bw4 = %s, bw5 = %s, bw6 = %s, bw7 = %s, bw8 = %s, bw9 = %s, '
            't0 = %s, t1 = %s, t2 = %s, t3 = %s, t4 = %s, t5 = %s, t6 = %s, t7 = %s, t8 = %s, t9 = %s, '
            'tw0 = %s, tw1 = %s, tw2 = %s, tw3 = %s, tw4 = %s, tw5 = %s, tw6 = %s, tw7 = %s, tw8 = %s, tw9 = %s '
            'WHERE movie_id = %s')

In [29]:
def set_table_analysis(val):
    try:
        connection = mysql.connector.connect(host=DB_credentials.host,
                                             database=DB_credentials.database,
                                             user=DB_credentials.user,
                                             password=DB_credentials.password)
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = connection.cursor()

            cursor.execute(create_table_query('positive_analysis'))
            cursor.execute(create_table_query('negative_analysis'))
            if len(val['pos_val_to_insert'])>0:
                cursor.executemany(add_table_query('positive_analysis'),val['pos_val_to_insert'])
            if len(val['neg_val_to_insert'])>0:
                cursor.executemany(add_table_query('negative_analysis'),val['neg_val_to_insert'])
            if len(val['pos_val_to_update'])>0:
                cursor.executemany(update_table_query('positive_analysis'),val['pos_val_to_update'])
            if len(val['neg_val_to_update'])>0:
                cursor.executemany(update_table_query('negative_analysis'),val['neg_val_to_update'])
            connection.commit()
            
            #query2 = ('SELECT * FROM positive_analysis '
            #          'JOIN movie ON positive_analysis.movie_id = movie.id;\n')
            # Create a pandas dataframe
            #pdff = pd.read_sql(query2, con=connection)

            cursor.close()
            connection.close()
            print("MySQL connection is closed")
            #print(pdff)

    except Error as e:
        print("Error while connecting to MySQL", e)


#mycursor = mydb.cursor()
#set_table_analysis(val,[])
                   


#mycursor.execute(sql, val)
#mycursor.execute("CREATE TABLE IF NOT EXISTS positive_analysis (id INT AUTO_INCREMENT PRIMARY KEY, name VARCHAR(255), address VARCHAR(255))")

In [30]:
def analyse_emotion_pipeline():
    # get movie ids to make analysis on
    movie_id_df = get_movie_id_list() 
    movie_id_list = movie_id_df["id"].values.tolist()
    analysis_id_df = get_analysis_id_list()
    analysis_id_list = analysis_id_df["movie_id"].values.tolist()
    print()

    for i in range(len(movie_id_list)//20):
        val ={
            'pos_val_to_insert' : [],
            'neg_val_to_insert' : [],
            'pos_val_to_update' : [],
            'neg_val_to_update' : []
        }
        for movie_id in movie_id_list[20*i:20*(i+1)]:
            if movie_id in analysis_id_list:
                pass
                #print("Movie with id "+str(movie_id) + " already analysed")
            else:
                print('Review analysis for movie with id : ' + str(movie_id))
                pdf = get_reviews_and_score_from_movie_id(movie_id)

                mySchema = StructType([ StructField("score", IntegerType(), True)\
                                       ,StructField("text", StringType(), True)])
                #Create DataFrame by changing schema
                df = spark.createDataFrame(pdf,schema=mySchema)

                pos_reviews=df.filter(col('score') >5)
                neg_reviews=df.filter(col('score') <=5)

                pos_text = pos_reviews.select('text').rdd
                neg_text = neg_reviews.select('text').rdd

                pos_output = analyse_reviews(pos_text,movie_id)
                neg_output = analyse_reviews(neg_text,movie_id)

            if movie_id in analysis_id_list:
                pass
                #print("Movie with id "+str(movie_id) + " already analysed")
                #list_columns_for_update = list_columns[1:] + list_columns[:1]
                #val['pos_val_to_update'].append(tuple([pos_output[x] for x in list_columns_for_update]))
                #val['neg_val_to_update'].append(tuple([neg_output[x] for x in list_columns_for_update]))
            else:
                val['pos_val_to_insert'].append(tuple([pos_output[x] for x in list_columns]))
                val['neg_val_to_insert'].append(tuple([neg_output[x] for x in list_columns]))



        set_table_analysis(val)
        
analyse_emotion_pipeline()
        
        

Connected to MySQL Server version  8.0.33


/tmp/ipykernel_20069/1775755409.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pdf = pd.read_sql(query, con=connection)


MySQL connection is closed
Connected to MySQL Server version  8.0.33


/tmp/ipykernel_20069/1775755409.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pdf = pd.read_sql(query, con=connection)


MySQL connection is closed

Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connec

...

Review analysis for movie with id : 18346
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Review analysis for movie with id : 18347
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Review analysis for movie with id : 18348
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Review analysis for movie with id : 18349
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Review analysis for movie with id : 18350
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Review analysis for movie with id : 18351
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Review analysis for movie with id : 18352
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Review analysis for movie with id : 18353
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Review analysis for movie with id : 18354
Connected to MySQL Server version  8.0.33
MySQL connection is closed
R

/tmp/ipykernel_20069/3622134320.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pdf = pd.read_sql(query, con=connection)


Review analysis for movie with id : 18373
Connected to MySQL Server version  8.0.33
MySQL connection is closed


/tmp/ipykernel_20069/3622134320.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pdf = pd.read_sql(query, con=connection)


Review analysis for movie with id : 18374
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Review analysis for movie with id : 18375
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Review analysis for movie with id : 18376
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Review analysis for movie with id : 18377
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Review analysis for movie with id : 18378
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Review analysis for movie with id : 18379
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Review analysis for movie with id : 18380
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Review analysis for movie with id : 18381
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Review analysis for movie with id : 18382
C

In [31]:
analyse_emotion_pipeline()

Connected to MySQL Server version  8.0.33
MySQL connection is closed


/tmp/ipykernel_20069/1775755409.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pdf = pd.read_sql(query, con=connection)


Connected to MySQL Server version  8.0.33
MySQL connection is closed



/tmp/ipykernel_20069/1775755409.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pdf = pd.read_sql(query, con=connection)


Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version  8.0.33
MySQL connection is closed
Connected to MySQL Server version 